# Семинар по pandas: merge и groupby

Не забудьте отправить решения задач в систему Яндекс.Контест:
- [Контест](https://contest.yandex.ru/contest/77201/enter) для 413 группы;
- [Контест](https://contest.yandex.ru/contest/77202/enter) для 414 группы;
- [Контест](https://contest.yandex.ru/contest/77203/enter) для 415 группы;
- [Контест](https://contest.yandex.ru/contest/77203/enter) для 416 группы;

В этом семинаре мы подробно разберем, как использовать функции `merge` и `groupby` в pandas. Также рассмотрим практические примеры работы с группами и модификации данных на их основе.

Перед началом установите библиотеку `kagglehub` и `tqdm`

In [ ]:
# Загрузка датасета с помощью kagglehub
import os
import pandas as pd
import kagglehub

# Загрузка последней версии датасета
path = kagglehub.dataset_download("ersany/online-retail-dataset")
print("Path to dataset files:", path)

# Чтение Excel-файла
df = pd.read_excel(os.path.join(path, "Online Retail.xlsx"))
df.head()

## Функция `pd.concat()`
`concat()` используется для объединения таблиц по строкам (`axis=0`) или по столбцам (`axis=1`).

Можно использовать, когда нужно просто "склеить" таблицы без условия по ключам.

In [ ]:
# Склеивание по строкам (axis=0)
df1 = pd.DataFrame({'ID': [1, 2], 'Value': ['A', 'B']})
df2 = pd.DataFrame({'ID': [3, 4], 'Value': ['C', 'D']})
display(df1)
display(df2)
res12 = pd.concat([df1, df2], axis=0)
res12


Правда теперь имеется нюанс - повторяющийся индекс. Иногда мы действительно хотим такое поведение. Напомним, что в первом семинаре при введении вас в объекты pandas вы узнали, что pd.Index - это мультимножество и допускает повторения.

In [ ]:
res12.loc[0]

А если мы не хотим дубликаты индексов?

Можно не дублировать, а игнорить индексы в каждом из объектов и собирать новый.

In [ ]:
# Объединение с повторяющимися индексами (можно сбросить индекс при необходимости)
pd.concat([df1, df2], axis=0, ignore_index=True)

Ну и столбцы тоже можно склеивать.

In [ ]:
# Склеивание по столбцам (axis=1)
df3 = pd.DataFrame({'A': [1, 2]})
df4 = pd.DataFrame({'B': ['X', 'Y']})
display(df3)
display(df4)
pd.concat([df3, df4], axis=1)


## Метод `.join()` в pandas
Метод `join()` используется для объединения DataFrame по их индексам.


In [ ]:
# Подготовим две таблицы для демонстрации
left = pd.DataFrame({
    'CustomerID': [1, 2, 3],
    'Name': ['Alice', 'Bob', 'Charlie']
})

right = pd.DataFrame({
    'CustomerID': [2, 3, 4],
    'Country': ['UK', 'Germany', 'France']
})

display(left)
display(right)

In [ ]:
# Пример использования join
left_indexed = left.set_index('CustomerID')
right_indexed = right.set_index('CustomerID')
display(left_indexed)
display(right_indexed)

joined = left_indexed.join(right_indexed, how='left')
joined

Здесь важно заметить, мы нигде явно не указали, как объединить наши данные. Так как оба имею совпадающий индекс, по которому и производится объединение.

## Функция `merge` — объединение таблиц
Позволяет производить объединения по различным столбцам.

В pandas функция `merge`, также как и `join`, позволяет использовать разные стратегии объединения таблиц: `inner`, `left`, `right`, `outer`.

![](media/join_types_sql.png)

In [ ]:
inner = pd.merge(left, right, left_on='CustomerID', right_on='CustomerID', how='inner')
inner

In [ ]:
# В случае, если ключи называются одинаково, можно указать его только один раз
inner = pd.merge(left, right, on='CustomerID', how='inner')
inner

In [ ]:
# LEFT JOIN: все строки из левой таблицы + совпадающие из правой
left_join = pd.merge(left, right, on='CustomerID', how='left')
left_join

In [ ]:
# RIGHT JOIN: все строки из правой таблицы + совпадающие из левой
right_join = pd.merge(left, right, on='CustomerID', how='right')
right_join

In [ ]:
# OUTER JOIN: все строки из обеих таблиц, пропущенные значения заменяются на NaN
outer = pd.merge(left, right, on='CustomerID', how='outer')
outer

### Разница между `merge()` и `join()`

Обе функции объединяют таблицы, но между ними есть важные различия:

#### `merge()` — универсальный инструмент
- Явно указываются ключевые столбцы с помощью `on`, `left_on`, `right_on`.
- Работает по значениям **столбцов**, а не индексов.
- Может объединять по разным столбцам в разных таблицах.
- Возвращает новый DataFrame с переиндексацией.
- Позволяет точно контролировать поведение при неуникальных ключах и коллизиях.

**Пример:**
```python
pd.merge(df1, df2, left_on='A', right_on='B', how='left')
```

---

#### `join()` — короче, но ограничен
- По умолчанию объединяет по **индексу правого DataFrame**.
- Удобен, если таблицы уже индексированы нужным образом.
- Менее гибкий: нельзя указать разные столбцы для объединения (как `left_on` и `right_on`).
- Может быть менее читаем в сложных кейсах.

**Пример:**
```python
df1.set_index('ID').join(df2.set_index('ID'))
```

---

#### Вывод:
- Используй `merge()` — когда нужна гибкость и точный контроль.
- Используй `join()` — если работаешь с индексами и хочешь короткий код.

## Функция `groupby` — группировка и агрегирование данных

In [ ]:
# Группируем по столбцу 'Country'
grouped = df.groupby('Country')

# Тип объекта
print("Тип объекта:", type(grouped))

# Получение списка групп
print("Группы:", grouped.groups.keys())

# Получение конкретной группы
group_uk = grouped.get_group('United Kingdom')
print("Размер группы 'United Kingdom':", group_uk.shape)

# Итерация по группам (ключ, подтаблица)
for name, group in list(grouped)[:2]:
    print(f"\nГруппа: {name}, Размер: {group.shape}")
    display(group.head(2))


In [ ]:
# Количество уникальных заказов по странам
df.groupby('Country')['InvoiceNo'].nunique().sort_values(ascending=False).head()

In [ ]:
# Общая сумма продаж по странам
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).head()

### Немного про агрегирующие функции (опять)

#### Метод `.agg()`

Метод `.agg()` (aggregate) используется после `groupby()` для применения одной или нескольких функций к колонкам.

In [ ]:
df.groupby('Country')['Quantity'].agg(sum).head()

In [ ]:
# Можно применять несколько функций одновременно
df.groupby('Country')['Quantity'].agg(['sum', 'mean', 'std']).head()

In [ ]:
# Можно применять разные функции к разным колонкам
df.groupby('Country').agg({
    'InvoiceNo': 'nunique',
    'Quantity': 'sum'
}).head()

In [ ]:
# Агрегация с использованием нескольких функций и колонок
df.groupby('Country').agg({
    'InvoiceNo': 'nunique',
    'Quantity': 'sum',
    'TotalPrice': ['sum', 'mean']
}).head()

In [ ]:
# Агрегация с использованием пользовательской функции
# На вход принимает Series, на выход - одно значение

def range_price(x):
    return x.max() - x.min()

df.groupby('Country').agg({
    'UnitPrice': range_price
}).head()

In [ ]:
df.groupby('Country').agg({
    "TotalPrice": [('SummedPrice', 'sum')],
    "UnitPrice": [('AvgPrice', 'mean')],
    "InvoiceNo": [('NumberOfUniqueOrders', 'nunique')]
}).head()


### Пример, почему группировка - круто.
Преобразование цен для каждого StockCode на 10% выше средней.

### Вариант 1: Цикл по уникальным StockCode

In [ ]:
df1 = df.copy()
for code in df1['StockCode'].unique():
    mask = df1['StockCode'] == code
    mean_price = df1.loc[mask, 'UnitPrice'].mean()
    df1.loc[mask, 'UnitPrice'] = mean_price * 1.1
df1[['StockCode', 'UnitPrice']].head()

Эта ячейка работает противно долго, прям ужасно. Её даже в образовательных целях запускать не хочется.

### Вариант 2: Используем groupby и преобразуем каждую группу

In [ ]:
df2 = df.copy()
df2['UnitPrice'] = df2.groupby('StockCode')['UnitPrice'].transform(lambda x: x.mean() * 1.1)
df2[['StockCode', 'UnitPrice']].head()

А вот это уже по нашенски. И быстро, и то что надо, без циклов, красиво, понятно, вообще здорово.

### Вариант 3: Используем apply для групп

In [ ]:
df3 = df.copy()

def adjust_price(group):
    # Функция принимает группу строк с одинаковым StockCode
    # и возвращает ту же группу с измененными значениями UnitPrice
    # Все строки в группе будут иметь UnitPrice, равный
    # средней цене группы, умноженной на 1.1
    group = group.copy()
    group['UnitPrice'] = group['UnitPrice'].mean() * 1.1
    return group

# group_keys=False - чтобы не создавать новый индекс
# почему важно - советую посмотреть в документации
df3 = df3.groupby('StockCode', group_keys=False).apply(adjust_price)
df3[['StockCode', 'UnitPrice']].head()

### Вариант 4: Используем tqdm и progress_apply

In [ ]:
from tqdm import tqdm
tqdm.pandas()

df4 = df.copy()
df4 = df4.groupby('StockCode', group_keys=False).progress_apply(adjust_price)
df4[['StockCode', 'UnitPrice']].head()

## Рубрика Домашка

Задачи:
- Изменяем цены
  - объединить страны в группы по первой букве страны (Australia, Austria; Belgium, Brazil и тд)
  - В группе с первой буквой "A" поднять цену на максимальную цену товарав группе
  - В группе с первой буквой "B" цену уменьшить на половину минимальной
  - В остальных группах цены увеличить на 10% от текущей цены объекта

- Сложные изменения цены
  - Сгрупируйте товары по странам
  - Каждую группу увеличьте в два раза по следующему правилу:
    - Увеличьте все цены в 2 раза и переименнуйте страну в country_name+"_bad_times"
    - Уменьшите все цены на 15% от средней цены (но чтоб цены были не меньше текущего минимума, все что меньше делаем равным текущему минимуму) и переименнуйте страну в country_name+"_great_times"

- Поиск акул рынка
  - Постройте отдельную таблицу пкупатель - число заказов `['CustomerID', 'NumOrders']`
  - Найдите самых активных (число заказов больше, чем у 95% всех покупателей)
  - Из основной таблицы выберете только информацию о наших акулах
  - Для каждого подсчитайте, сколько суммарно в каждой из стран было потрачено $ (оформить в виде таблицы с колонками CustomerID и нразваниями стран)
  - Найти гениев, которые покупали в нескольких странах
  - (подсказка: unstack, reset_index, columns.name = None)

In [ ]:
# Если не понятно что и как делать - спрашивайте